In [1]:
import pandas as pd

In [13]:
fips_url_full = 'https://www.dropbox.com/s/04eu0q6sgph8wto/US_FIPS_Codes.xls?dl=1'
fips_url_abv = 'https://www.dropbox.com/s/d503mesxlsc1yfa/county_fips.csv?dl=1'
fl_pop_path = '../20_intermediate_files/fl_shipment_pop.csv'
fl_path = '../20_intermediate_files/florida_shipment_cleaned.csv'

In [3]:
fl_df = pd.read_csv(fl_path)
fl_df.head()

,BUYER_STATE,BUYER_COUNTY,YEAR,MME
0,AZ,APACHE,2006,4.956969e+06
1,AZ,APACHE,2007,5.611684e+06
2,AZ,APACHE,2008,5.853244e+06
3,AZ,APACHE,2009,7.344332e+06
4,AZ,APACHE,2010,7.732743e+06


In [4]:
fips_full = pd.read_excel(fips_url_full, header=1, dtype={'State':str, 'County Name':str,'FIPS State':str, 'FIPS County':str})
fips_full['FIPS Code'] = fips_full['FIPS State'] + fips_full['FIPS County']
fips_full['FIPS Code'] = fips_full['FIPS Code'].str.lstrip('0')
fips_full.drop(columns=['FIPS State', 'FIPS County'], inplace=True)
fips_full['FIPS Code'] = fips_full['FIPS Code'].astype(int)
fips_full.head()

,State,County Name,FIPS Code
0,Alabama,Autauga,1001
1,Alabama,Baldwin,1003
2,Alabama,Barbour,1005
3,Alabama,Bibb,1007
4,Alabama,Blount,1009


In [5]:
fips_abv = pd.read_csv(fips_url_abv)
fips_abv.rename(columns={'countyfips': 'FIPS Code'}, inplace=True)
fips_abv

,BUYER_COUNTY,BUYER_STATE,FIPS Code
0,AUTAUGA,AL,1001
1,BALDWIN,AL,1003
2,BARBOUR,AL,1005
3,BIBB,AL,1007
4,BLOUNT,AL,1009
...,...,...,...
3138,WASHAKIE,WY,56043
3139,WESTON,WY,56045
3140,SKAGWAY,AK,2230
3141,HOONAH ANGOON,AK,2105


In [6]:
fips = fips_abv.merge(fips_full, how = 'left', on=['FIPS Code'])
fips.drop(columns=['County Name'],inplace=True)
#NaN for AK
fips.isna().sum()

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           3
dtype: int64

In [7]:
missing_state = fips['State'].isna()
missing_state

0       False
1       False
2       False
3       False
4       False
        ...  
3138    False
3139    False
3140     True
3141     True
3142     True
Name: State, Length: 3143, dtype: bool

In [8]:
impute_dict=dict({'AK':'Arkansas'})
fips.loc[missing_state, 'State'] = fips.loc[missing_state, 'BUYER_STATE'].map(impute_dict)
print(fips.isna().sum())
fips.shape

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           0
dtype: int64


(3143, 4)

In [9]:
fl_ship_fips = fl_df.merge(fips, how='left', on=['BUYER_STATE','BUYER_COUNTY'])
fl_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina


In [14]:
fl_df_pop = pd.read_csv(fl_pop_path)
fl_df_pop['STNAME'].unique()


array(['Arizona', 'Florida', 'Louisiana', 'South Carolina'], dtype=object)

In [16]:
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace(' County', '')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.upper()
fl_df_pop

,STNAME,CTYNAME,YEAR,POPULATION
0,Arizona,APACHE,2000,69516.0
1,Arizona,COCHISE,2000,118028.0
2,Arizona,COCONINO,2000,116717.0
3,Arizona,GILA,2000,51355.0
4,Arizona,GRAHAM,2000,33541.0
...,...,...,...,...
3855,South Carolina,SPARTANBURG,2019,320254.0
3856,South Carolina,SUMTER,2019,106635.0
3857,South Carolina,UNION,2019,27221.0
3858,South Carolina,WILLIAMSBURG,2019,30256.0


In [21]:
fl_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina


In [22]:
fl_ship_pop = fl_ship_fips.merge(fl_df_pop, how='left', left_on=['State', 'BUYER_COUNTY','YEAR'], right_on=['STNAME', 'CTYNAME', 'YEAR'])

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona,Arizona,APACHE,69066.0
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona,Arizona,APACHE,69278.0
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona,Arizona,APACHE,69520.0
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona,Arizona,APACHE,70591.0
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona,Arizona,APACHE,71828.0
...,...,...,...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina,South Carolina,YORK,226871.0
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina,South Carolina,YORK,230131.0
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina,South Carolina,YORK,234151.0
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina,South Carolina,YORK,238736.0
